In [88]:
import tensorflow as tf
import pandas as pd
import keras
import numpy as np
import os
import matplotlib.pyplot as plt

In [89]:
a=tf.range(10)
a1=tf.data.Dataset.from_tensor_slices(a)
a1

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [90]:
for item in a1:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [91]:
a=tf.Variable([[12,23,34],[45,36,23]])
a

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[12, 23, 34],
       [45, 36, 23]])>

In [92]:
a1=tf.data.Dataset.from_tensor_slices(a)
for item in a1:
    print(item)

tf.Tensor([12 23 34], shape=(3,), dtype=int32)
tf.Tensor([45 36 23], shape=(3,), dtype=int32)


In [93]:
a=tf.range(10)
dataset=tf.data.Dataset.from_tensor_slices(a)

In [94]:
dataset_batch=dataset.repeat(5).batch(7,drop_remainder=True)
for item in dataset_batch:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9 0 1 2 3 4], shape=(7,), dtype=int32)
tf.Tensor([5 6 7 8 9 0 1], shape=(7,), dtype=int32)
tf.Tensor([2 3 4 5 6 7 8], shape=(7,), dtype=int32)


# lets do some filter of the data

In [95]:
dataset_batch=dataset_batch.map(lambda x:x*2)
for item in dataset_batch:
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([16 18  0  2  4  6  8], shape=(7,), dtype=int32)
tf.Tensor([10 12 14 16 18  0  2], shape=(7,), dtype=int32)
tf.Tensor([ 4  6  8 10 12 14 16], shape=(7,), dtype=int32)


In [96]:
dataset_batch=dataset_batch.unbatch()

In [97]:
dataset_batch.filter(lambda x:x<10)

<_FilterDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [98]:
for item in dataset_batch.take(5):
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)


In [99]:
data_shufle=tf.data.Dataset.range(10).repeat(3)
data_shufle=data_shufle.shuffle(buffer_size=3,seed=42).batch(7)
for item in data_shufle:
    print(item)

tf.Tensor([1 3 0 4 2 5 6], shape=(7,), dtype=int64)
tf.Tensor([8 7 1 0 3 2 5], shape=(7,), dtype=int64)
tf.Tensor([4 6 9 8 9 7 0], shape=(7,), dtype=int64)
tf.Tensor([3 1 4 5 2 8 7], shape=(7,), dtype=int64)
tf.Tensor([6 9], shape=(2,), dtype=int64)


In [100]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

housing_df=fetch_california_housing()
x_train_full,x_test,y_train_full,y_test=train_test_split(housing_df.data,housing_df.target.reshape(-1,1),random_state=42)
x_train,x_valid,y_train,y_valid=train_test_split(x_train_full,y_train_full)

In [101]:
housing_df.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [102]:
standard=StandardScaler()
standard.fit(x_train)

StandardScaler()

In [103]:
x_mean=standard.mean_
x_std=standard.scale_

In [104]:
train_data=np.c_[x_train,y_train]
valid_data=np.c_[x_valid,y_valid]
test_data=np.c_[x_test,y_test]

In [105]:
train_data.shape,valid_data.shape,test_data.shape

((11610, 9), (3870, 9), (5160, 9))

In [106]:
header_cols=housing_df.feature_names+["MedianHouseValue"]

In [107]:
header=",".join(header_cols)

In [108]:
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = os.path.join("datasets", "housing")
    os.makedirs(housing_dir, exist_ok=True)
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")

    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

In [109]:
# lets define a function for spliting our data into multiple files
train_filepath=save_to_multiple_csv_files(train_data,"train",header,n_parts=20)
valid_filepath=save_to_multiple_csv_files(valid_data,"valid",header,n_parts=20)
test_filepath=save_to_multiple_csv_files(test_data,"test",header,n_parts=10)

In [110]:
pd.read_csv(train_filepath[0]).head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,3.5214,15.0,3.049945,1.106548,1447.0,1.605993,37.63,-122.43,1.442
1,5.3275,5.0,6.490060,0.991054,3464.0,3.443340,33.69,-117.39,1.687
2,3.1000,29.0,7.542373,1.591525,1328.0,2.250847,38.44,-122.98,1.621
3,7.1736,12.0,6.289003,0.997442,1054.0,2.695652,33.55,-117.70,2.621
4,2.0549,13.0,5.312457,1.085092,3297.0,2.244384,33.93,-116.93,0.956


In [111]:
with open(train_filepath[0]) as f:
    for i in range(5):
        print(f.readline(),end=" ")

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
 3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442
 5.3275,5.0,6.490059642147117,0.9910536779324056,3464.0,3.4433399602385686,33.69,-117.39,1.687
 3.1,29.0,7.5423728813559325,1.5915254237288134,1328.0,2.2508474576271187,38.44,-122.98,1.621
 7.1736,12.0,6.289002557544757,0.9974424552429667,1054.0,2.6956521739130435,33.55,-117.7,2.621
 

In [112]:
train_filepath

['datasets\\housing\\my_train_00.csv',
 'datasets\\housing\\my_train_01.csv',
 'datasets\\housing\\my_train_02.csv',
 'datasets\\housing\\my_train_03.csv',
 'datasets\\housing\\my_train_04.csv',
 'datasets\\housing\\my_train_05.csv',
 'datasets\\housing\\my_train_06.csv',
 'datasets\\housing\\my_train_07.csv',
 'datasets\\housing\\my_train_08.csv',
 'datasets\\housing\\my_train_09.csv',
 'datasets\\housing\\my_train_10.csv',
 'datasets\\housing\\my_train_11.csv',
 'datasets\\housing\\my_train_12.csv',
 'datasets\\housing\\my_train_13.csv',
 'datasets\\housing\\my_train_14.csv',
 'datasets\\housing\\my_train_15.csv',
 'datasets\\housing\\my_train_16.csv',
 'datasets\\housing\\my_train_17.csv',
 'datasets\\housing\\my_train_18.csv',
 'datasets\\housing\\my_train_19.csv']

In [113]:
valid_filepath

['datasets\\housing\\my_valid_00.csv',
 'datasets\\housing\\my_valid_01.csv',
 'datasets\\housing\\my_valid_02.csv',
 'datasets\\housing\\my_valid_03.csv',
 'datasets\\housing\\my_valid_04.csv',
 'datasets\\housing\\my_valid_05.csv',
 'datasets\\housing\\my_valid_06.csv',
 'datasets\\housing\\my_valid_07.csv',
 'datasets\\housing\\my_valid_08.csv',
 'datasets\\housing\\my_valid_09.csv',
 'datasets\\housing\\my_valid_10.csv',
 'datasets\\housing\\my_valid_11.csv',
 'datasets\\housing\\my_valid_12.csv',
 'datasets\\housing\\my_valid_13.csv',
 'datasets\\housing\\my_valid_14.csv',
 'datasets\\housing\\my_valid_15.csv',
 'datasets\\housing\\my_valid_16.csv',
 'datasets\\housing\\my_valid_17.csv',
 'datasets\\housing\\my_valid_18.csv',
 'datasets\\housing\\my_valid_19.csv']

# building a pipeline

In [114]:
filepath_dataset=tf.data.Dataset.list_files(train_filepath)

In [115]:
for filepath in filepath_dataset:
    print(filepath)

tf.Tensor(b'datasets\\housing\\my_train_15.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_08.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_10.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_12.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_14.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_17.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_06.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_03.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_00.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_05.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_19.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_09.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_07.csv', sh

In [116]:
n_readers=5
dataset=filepath_dataset.interleave(
    lambda filepath:tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers
)

In [117]:
for line in dataset.take(5):
    print(line)
print(type(line))

tf.Tensor(b'3.6641,17.0,5.577142857142857,1.1542857142857144,511.0,2.92,40.85,-121.07,0.808', shape=(), dtype=string)
tf.Tensor(b'2.4792,24.0,3.4547038327526134,1.1341463414634145,2251.0,3.921602787456446,34.18,-118.38,2.0', shape=(), dtype=string)
tf.Tensor(b'4.2708,45.0,5.121387283236994,0.953757225433526,492.0,2.8439306358381504,37.48,-122.19,2.67', shape=(), dtype=string)
tf.Tensor(b'3.6875,44.0,4.524475524475524,0.993006993006993,457.0,3.195804195804196,34.04,-118.15,1.625', shape=(), dtype=string)
tf.Tensor(b'2.1856,41.0,3.7189873417721517,1.0658227848101265,803.0,2.0329113924050635,32.76,-117.12,1.205', shape=(), dtype=string)
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [118]:
record_defaults=[0, np.nan, tf.constant(np.nan, dtype=tf.float64), "Hello", tf.constant([])]
parsed_fields = tf.io.decode_csv('1,2,3,4,5', record_defaults)
parsed_fields

[<tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.0>,
 <tf.Tensor: shape=(), dtype=float64, numpy=3.0>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'4'>,
 <tf.Tensor: shape=(), dtype=float32, numpy=5.0>]

In [119]:
parsed_fields=tf.io.decode_csv(",,,,5",record_defaults)
parsed_fields

[<tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=nan>,
 <tf.Tensor: shape=(), dtype=float64, numpy=nan>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'Hello'>,
 <tf.Tensor: shape=(), dtype=float32, numpy=5.0>]

In [120]:
try:
    parsed_fields = tf.io.decode_csv(',,,,', record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

{{function_node __wrapped__DecodeCSV_OUT_TYPE_5_device_/job:localhost/replica:0/task:0/device:CPU:0}} Field 4 is required but missing in record 0! [Op:DecodeCSV]


In [121]:
try:
    parsed_fields = tf.io.decode_csv('1,2,3,4,5,6,7', record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

{{function_node __wrapped__DecodeCSV_OUT_TYPE_5_device_/job:localhost/replica:0/task:0/device:CPU:0}} Expect 5 fields but have 7 in record 0 [Op:DecodeCSV]


In [122]:
[0.]*4

[0.0, 0.0, 0.0, 0.0]

In [123]:
[0.]*4 + [12,23]

[0.0, 0.0, 0.0, 0.0, 12, 23]

In [124]:
n_inputs=8
def preprocess(line):
    defaults=[0.]*n_inputs +[tf.constant([],dtype=tf.float32)]
    changed=tf.io.decode_csv(line,defaults)
    x=tf.stack(changed[:-1])
    y=tf.stack(changed[-1:])
    return x-x_mean/x_std,y

In [125]:
preprocess(b'3.1821,9.0,5.675156174915906,1.0480538202787122,6716.0,3.2272945699183087,34.44,-117.35,0.956')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([ 1.1437545e+00,  6.7355642e+00,  3.5358925e+00, -1.3144144e+00,
         6.7147002e+03,  1.9742706e+00,  1.7740389e+01, -5.7585697e+01],
       dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.956], dtype=float32)>)

In [126]:
def csv_reader_dataset(filepath,repeat=1,n_readers=5,n_read_threads=None,
                      shuffle_buffer_size=1000,n_parse_threads=5,batch_size=32):
    dataset=tf.data.Dataset.list_files(filepath)
    dataset=dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).repeat(1),
        cycle_length=n_readers,num_parallel_calls=n_read_threads
    )
    dataset=dataset.shuffle(shuffle_buffer_size)
    dataset=dataset.map(preprocess,num_parallel_calls=n_parse_threads)
    dataset=dataset.batch(batch_size)
    return dataset.prefetch(1)

In [127]:
tf.random.set_seed(42)

In [128]:
train_sample=csv_reader_dataset(train_filepath,batch_size=32)
for x_batch,y_batch in train_sample.take(2):
    print("x: ",x_batch)
    print()
    print("y :",x_batch)

x:  tf.Tensor(
[[-8.1954563e-01  3.4735565e+01  2.2320814e+00 -1.1080823e+00
   8.6370020e+02  1.2762156e+00  1.7230391e+01 -5.7115696e+01]
 [-4.3364561e-01  7.7355642e+00  2.7277312e+00 -1.2655882e+00
   1.8517002e+03  1.7896688e+00  1.7880392e+01 -5.7645702e+01]
 [ 1.5178545e+00  4.0735565e+01  2.8945827e+00 -1.3655452e+00
   7.9070020e+02  1.1838989e+00  2.0280390e+01 -6.2235699e+01]
 [ 7.0416546e+00  1.5735564e+01  6.7863564e+00 -1.3542038e+00
   3.8070020e+02  1.9040008e+00  2.0510389e+01 -6.2105701e+01]
 [ 1.1295438e-01  5.7355642e+00  5.9785819e+00 -8.0354571e-01
   7.8470020e+02  1.3934405e+00  2.3910391e+01 -6.1905697e+01]
 [ 2.1116545e+00  1.1735564e+01  4.3783145e+00 -1.2472339e+00
   1.3837002e+03  1.4520540e+00  1.6530390e+01 -5.7315701e+01]
 [ 1.0449545e+00  2.4735563e+01  1.4191363e+00 -1.3736682e+00
   1.5927002e+03  1.2973759e+00  1.7320391e+01 -5.8645702e+01]
 [-5.3834558e-01  2.9735563e+01  2.8116798e+00 -1.1737890e+00
   5.9870020e+02  1.0111268e+00  2.0770391e+01 -

In [129]:
train_set=csv_reader_dataset(train_filepath)
valid_set=csv_reader_dataset(valid_filepath)
test_set=csv_reader_dataset(test_filepath)

In [130]:
# lets train our model
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)